In [107]:
import numpy as np
import cv2

# 定義手動選擇的影像點和對應的世界點
image_points = [
    np.array([[630, 349], [686, 1024], [1375, 737], [1441, 176]]),  #  1
    np.array([[357, 160], [437, 711], [686, 1024], [630, 349]]),   #  2
    np.array([[357, 160], [630, 349], [1441, 176], [1050, 57]])    #  3
]

world_points = [
    np.array([[0, 0], [0, 6], [8, 6], [8, 0]]),  # 矩形+ 1
    np.array([[0, 0], [0, 6], [6, 6], [6, 0]]),  # 矩形 2
    np.array([[0, 0], [0, 6], [8, 6], [8, 0]])   # 矩形 3
]


def compute_homography(image_pts, world_pts):
    """計算單應性矩陣。"""
    H, status = cv2.findHomography(world_pts, image_pts)
    return H

def calculate_intrinsic_matrix(homographies):
    """從一組單應性矩陣中計算內參矩陣 K。"""
    n = len(homographies)
    V = np.zeros((2 * n, 6))

    def v_ij(h, i, j):
        return np.array([
            h[0, i] * h[0, j], h[0, i] * h[1, j] + h[1, i] * h[0, j],
            h[1, i] * h[1, j], h[2, i] * h[0, j] + h[0, i] * h[2, j],
            h[2, i] * h[1, j] + h[1, i] * h[2, j], h[2, i] * h[2, j]
        ])

    # 根據單應性矩陣的約束填充 V 矩陣
    for i, H in enumerate(homographies):
        V[2 * i] = v_ij(H, 0, 1)
        V[2 * i + 1] = v_ij(H, 0, 0) - v_ij(H, 1, 1)

    # 解線性系統
    _, _, vh = np.linalg.svd(V)
    b = vh[-1]

    # 從 b 中恢復內參矩陣 K
    B11, B12, B22, B13, B23, B33 = b
    v0 = (B12 * B13 - B11 * B23) / (B11 * B22 - B12 ** 2)
    lambda_ = B33 - (B13 ** 2 + v0 * (B12 * B13 - B11 * B23)) / B11
    alpha = np.sqrt(lambda_ / B11)
    beta = np.sqrt(lambda_ * B11 / (B11 * B22 - B12 ** 2))
    # gamma = -B12 * alpha ** 2 * beta / lambda_
    gamma = -B12 * alpha ** 2 / lambda_

    u0 = gamma * v0 / beta - B13 * alpha ** 2 / lambda_

    K = np.array([
        [alpha, gamma, u0],
        [0, beta, v0],
        [0, 0, 1]
    ])

    return K

def compute_camera_extrinsic(H, K):
    """計算外參數（旋轉矩陣 R 和平移向量 t）。"""
    K_inv = np.linalg.inv(K)
    extrinsic = K_inv @ H

    # 正規化以確保旋轉列的適當尺度
    norm_R1 = np.linalg.norm(extrinsic[:, 0])
    norm_R2 = np.linalg.norm(extrinsic[:, 1])
    R1 = extrinsic[:, 0] / norm_R1
    R2 = extrinsic[:, 1] / norm_R2

    # 計算旋轉矩陣的第三列作為叉積
    R3 = np.cross(R1, R2)
    R2 = np.cross(R3, R1)

    R = np.column_stack((R1, R2, R3))
    t = extrinsic[:, 2] / norm_R1
    return R, t

def verify_translation_vector(R, t, world_pts, image_pts, K):
    """通過將世界點投影回影像點來驗證平移向量。"""
    projected_points = []
    for pt in world_pts:
        pt_homogeneous = np.append(pt, 1)
        projected_pt = K @ (R @ pt_homogeneous + t)
        projected_pt /= projected_pt[2]
        projected_points.append(projected_pt[:2])

    projected_points = np.array(projected_points)
    print("投影點:\n", projected_points)
    print("原始影像點:\n", image_pts)
    return projected_points

# 計算每組影像點和世界點的單應性矩陣
homographies = [compute_homography(img_pts, wld_pts) for img_pts, wld_pts in zip(image_points, world_points)]

# 計算內參矩陣 K
K = calculate_intrinsic_matrix(homographies)

# 計算第三個單應性矩陣的外參數
R_top, t_top = compute_camera_extrinsic(homographies[2], K)

# 顛倒旋轉矩陣的 x 和 z 軸
R_top[:, 0] = -R_top[:, 0]
R_top[:, 2] = -R_top[:, 2]

# 調整平移向量到世界座標
t_world = np.array([-t_top[0] + 8, t_top[1], -t_top[2] + 6])
# t_world = np.array([-t_top[0] , t_top[1], t_top[2] ])

# 通過將世界點投影回影像點來驗證平移向量
# A = verify_translation_vector(R_top, t_world, world_points[2], image_points[2], K)

# 結果
print("內參矩陣 (K):")
print(K)
print("\n旋轉矩陣 (R):")
print(R_top)
print("\n平移向量 (t):")
print(t_world)


內參矩陣 (K):
[[1.89667447e+03 2.32413279e-03 9.55255233e+02]
 [0.00000000e+00 1.87967155e+03 5.55294896e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

旋轉矩陣 (R):
[[-0.83418053  0.55148932 -0.00153087]
 [ 0.25153461  0.37799695 -0.89098181]
 [-0.49078832 -0.74362475 -0.45403635]]

平移向量 (t):
[ 13.59174866  -3.72815129 -11.72777739]
